In [ ]:
import psycopg2
import pandas as pd
import geopandas as gpd
import pyproj
from pylab import *
import pdb

#to download derbyshire data, convert to routing format, and insert into pgrouting database:
#wget http://api.openstreetmap.org/api/0.6/map?bbox=-1.4563,53.2478,-1.4011,53.2767 -O ~/data/osm/dcc.osm
#osm2pgrouting  -f ~/data/osm/dcc.osm   -d dcc  -U charles  -W charles

con = psycopg2.connect(database='dcc', user='charles')
cur = con.cursor()

#df_sites = gpd.GeoDataFrame.from_file('/home/charles/data/msc/dcc/examples/BluetoothUnits.shp') #in bng
df_roads = gpd.GeoDataFrame.from_file('/home/charles/data/osm/dcc.shp/lines.shp') #as WGS84 latlon
wgs84  = pyproj.Proj(init='epsg:4326')  #WGS84
bng    = pyproj.Proj(init='epsg:27700') #british national grid

#routing - gives a list of edge gids (not osm_ids, which may be split up into smaller gids by osm2pgr)
sql = "SELECT * FROM pgr_dijkstra('SELECT gid AS id,source,target,length AS cost FROM ways', 170,750, directed := false), ways WHERE ways.gid=pgr_dijkstra.edge;"
df_route = gpd.GeoDataFrame.from_postgis(sql,con,geom_col='the_geom')
b_route=dict()
routeEdgesOsmIDs = df_route['edge']
for i in routeEdgesOsmIDs:
    b_route[i]=1

if 1:
    #plot roads
    sql = "SELECT * FROM ways;"
    df_roads = gpd.GeoDataFrame.from_postgis(sql,con,geom_col='the_geom')

    pdb.set_trace()

    for i in range(0,df_roads.shape[0]): 
        road = df_roads.iloc[i]
        lons = road['the_geom'].coords.xy[0] #coordinates in latlon
        lats = road['the_geom'].coords.xy[1]
        gid = int(road.gid) 

        xs=[]
        ys=[]
        n_segments = len(lons)
        for j in range(0, n_segments):
            (x,y) = pyproj.transform(wgs84, bng, lons[j], lats[j]) #project to BNG -- uses nonISO lonlat convention 
            xs.append(x)
            ys.append(y)
        color='y'
        if gid in b_route:  #color the route in red, other roads yellow
            color='r'
        plot(xs, ys, color)  

sql = "SELECT * from sensorsite;"
df_sites = pd.read_sql_query(sql, con)
plot(df_sites.easting,df_sites.northing, 'bo')
